In [378]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import itertools
import csv
import os
import re
from requests import get

import unicodedata
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

# import acquire
# from acquire import get_article_text

In [169]:
# def get_article_text():
#     blobs = ['elixer-ecto/ecto', 'microsoft/calculator', 'koalaman/shellcheck', 'mjbahmani/Kaggle-Competitions']
    
#     good_blobs = []
#     read_me_list = []
    
#     for blob in blobs:
    
#         url_1 = 'https://github.com/' + str(blob) + '/blob/master/README.md'
#         url = url_1

#         response = requests.get(url)
#         soup = BeautifulSoup(response.text)
#         read_me = soup.find(class_='markdown-body entry-content p-3 p-md-6')
        
#         if read_me != None:
        
#             read_me_list.append(read_me.text)
#             good_blobs.append(str(blob))
            
#     return pd.DataFrame(list(zip(good_blobs, read_me_list)))

In [299]:
# get_article_text()

In [187]:
blob_list = [['microsoft / Terminal'],
 ['jackfrued / Python-100-Days'],
 ['kkuchta / css-only-chat'],
 ['microsoft / PowerToys'],
 ['jolaleye / cssfx'],
 ['MisterBooo / LeetCodeAnimation'],
 ['flutter / flutter_web'],
 ['TheAlgorithms / Python'],
 ['hiroppy / fusuma'],
 ['CyC2018 / CS-Notes'],
 ['jaywcjlove / linux-command'],
 ['flutter / flutter'],
 ['996icu / 996.ICU'],
 ['STVIR / pysot'],
 ['minamarkham / formation'],
 ['azl397985856 / leetcode'],
 ['qianguyihao / Web'],
 ['microsoft / react-native-windows'],
 ['sql-machine-learning / sqlflow'],
 ['sabakkps / backslide'],
 ['dgryski / go-perfbook'],
 ['Snailclimb / JavaGuide'],
 ['microsoft / vscode'],
 ['markphelps / flipt'],
 ['teoga / awesome-product-design']]

In [198]:
blob_list = [['microsoft / Terminal'],
 ['jackfrued / Python-100-Days'],
 ['kkuchta / css-only-chat'],
 ['microsoft / PowerToys'],
 ['jolaleye / cssfx'],
 ['MisterBooo / LeetCodeAnimation']]

## Changing the acquire function to take a list of lists.

In [398]:

def get_url_list():
    urls = []
    headers = {'User-Agent': 'Student'}
    url = 'https://github.com/search?o=desc&p=1&q=stars%3A%3E1&s=updated&type=Repositories'
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    max_page = 3
    for i in range(1,max_page):
        url = 'https://github.com/search?o=desc&p=' + str(i) + '&q=stars%3A%3E1&s=updated&type=Repositories/'
#         print(f'traversing url: {url}')
        response = get(url, headers=headers)
        soup = BeautifulSoup(response.text)
          
        repo = soup.findAll('li', class_='repo-list-item d-flex flex-column flex-md-row flex-justify-start py-4 public source')
        for div in repo:
            one = []
            one.append(re.findall(r'.*/.*',div.text)[0])
            urls.append(one)
#         print(len(urls))
#         print(urls)
    final_urls = []
    for url in urls:
        url = 'https://github.com/'+url[0]
        final_urls.append(url)
        
        
    return final_urls

In [399]:
url_list = get_url_list()

In [400]:
url_list

['https://github.com/ltoddy/resume',
 'https://github.com/jinchuika/app-suni',
 'https://github.com/iotaledger/documentation',
 'https://github.com/aroberge/friendly-traceback',
 'https://github.com/duo-labs/webauthn',
 'https://github.com/carla-simulator/carla',
 'https://github.com/Happyxianyueveryday/myds',
 'https://github.com/uktrade/data-hub-leeloo',
 'https://github.com/srgrr/TFM',
 'https://github.com/floscr/Dotfiles',
 'https://github.com/floscr/Dotfiles',
 'https://github.com/zooniverse/front-end-monorepo',
 'https://github.com/Sara-HY/sara-hy.github.io',
 'https://github.com/hysophie/project_dynamic',
 'https://github.com/kokolihapihvi/RockSnifferLib',
 'https://github.com/teambit/bit-javascript',
 'https://github.com/BBVA/kapow',
 'https://github.com/Creuna-Oslo/react-scripts',
 'https://github.com/nim-lang/Nim',
 'https://github.com/vrhk/vrhk.github.io']

In [401]:
url_list[0].split('.com/')[1]

'ltoddy/resume'

In [402]:
url_list[0]

'https://github.com/ltoddy/resume'

In [423]:
trending_url = 'https://github.com/trending'
trend_response = requests.get(trending_url)

soup = BeautifulSoup(trend_response.content, 'html.parser')

repo = soup.findAll('li', class_='col-12 d-block width-full py-4 border-bottom')
print(type(repo))
def repo_name_and_label(bs4resultset):
    name_label = []
    for div in bs4resultset:
        one = []
        one.append(re.findall(r'.*\s/\s.*',div.text)[0])
        lang = list(set(re.findall(r'.+', div.text.lower())) & set(langs))
        if lang != []:
            one.append(lang[0])
        else:
            one.append('na')
        name_label.append(one)
    return name_label

<class 'bs4.element.ResultSet'>


In [417]:
repo_name_and_label(repo)

[]

In [411]:
def get_article_text():
    urls = list(url_list)
    good_blobs = []
    read_me_list = []
    
    for url in urls:
    
        blob_string=url.split('.com/')[1]
        blob_string=blob_string.replace(' ', '')
        url_1 = str(url) + '/blob/master/README.md'
        url = url_1.replace(' ', '')

        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        read_me = soup.find(class_='markdown-body entry-content p-3 p-md-6')
    #       read_me_list.append(str(blob) + ':')

        if read_me != None:

            read_me_list.append(read_me.text)
            good_blobs.append(str(blob_string))

    return pd.DataFrame(list(zip(good_blobs, read_me_list)))

In [412]:
get_article_text()

,0,1
0,ltoddy/resume,\n\n\n\n\n\n\nltoddy's resume\nUsage:\nrun at ...
1,jinchuika/app-suni,SUNI2\nde Fundación Sergio Paiz Andrade\nSiste...
2,iotaledger/documentation,IOTA Documentation\nThis repository contains u...
3,aroberge/friendly-traceback,friendly-traceback\nAimed at Python beginners:...
4,duo-labs/webauthn,WebAuthn Library\n\n\n\nThis library is meant ...
5,carla-simulator/carla,CARLA Simulator\n\n\n\n\n\n\nCARLA is an open-...
6,Happyxianyueveryday/myds,myds\n(myds = my data-structure)\n5月1日开始更新，目标不...
7,uktrade/data-hub-leeloo,Data Hub Leeloo\n\n\n\n\nLeeloo provides an AP...
8,srgrr/TFM,Master's Thesis Repository\nThis repository co...
9,floscr/Dotfiles,"Dotfiles\nMy configuration files for vim, tmux..."


In [188]:
# def get_article_text():
# #     blobs = ['elixer-ecto/ecto', 'microsoft/calculator', 'koalaman/shellcheck', 'mjbahmani/Kaggle-Competitions']
#     blobs = blob_list
#     good_blobs = []
#     read_me_list = []
    
#     for blob in blobs:
#         blob_string=blob[0]
#         blob_string=blob_string.replace(' ', '')
#         url_1 = 'https://github.com/' + str(blob_string) + '/blob/master/README.md'
#         url = url_1

#         response = requests.get(url)
#         soup = BeautifulSoup(response.text)
#         read_me = soup.find(class_='markdown-body entry-content p-3 p-md-6')
# #         read_me_list.append(str(blob) + ':')
        
#         if read_me != None:
        
#             read_me_list.append(read_me.text)
#             good_blobs.append(str(blob))
        
#     return pd.DataFrame(list(zip(good_blobs, read_me_list)))

In [192]:
df = get_article_text()

## Did something wrong happen?  There are a lot of non English characters.  A manual inspection of this readme confirms this is what's in it.  Later on, we'll run a 'basic clean'.  We'll look at the % of words dropped.  If a document returns with only 50-60% of it's original words, it might contain a lot of items that are not considered English.

In [193]:
df[1][1]

"Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形势分析\n简单的说，Python是一个“优雅”、“明确”、“简单”的编程语言。\n\n学习曲线低，非专业人士也能上手\n开源系统，拥有强大的生态圈\n解释型语言，完美的平台可移植性\n支持面向对象和函数式编程\n能够通过调用C/C++代码扩展功能\n代码规范程度高，可读性强\n\n目前几个比较流行的领域，Python都有用武之地。\n\n云基础设施 - Python / Java / Go\nDevOps - Python / Shell / Ruby / Go\n网络爬虫 - Python / PHP / C++\n数据分析挖掘 - Python / R / Scala / Matlab\n机器学习 - Python / R / Java / Lisp\n\n作为一名Python开发者，主要的就业领域包括：\n\nPython服务器后台开发 / 游戏服务器开发 / 数据接口开发工程师\nPython自动化运维工程师\nPython数据分析 / 数据可视化 / 大数据工程师\nPython爬虫工程师\nPython聊天机器人开发 / 图像识别和视觉算法 / 深度学习工程师\n\n下图显示了主要城市Python招聘需求量及薪资待遇排行榜（截止到2018年5月）。\n\n\n\n给初学者的几个建议：\n\nMake English as your working language.\nPractice makes perfect.\nAll experience comes from mistakes.\nDon't be one of the leeches.\nEither stand out or kicked out.\n\nDay01~15 - Python语言基础\nDay01 - 初识Python\n\nPython简介 - Python的历史 / Python的优缺点 / Python的应用领域\n搭建编程环境 - Windows环境 / Linux环境 / MacOS环境\n从终端运行Python程序 - DOS命令 / Hello, world / print函数 / 运行程序\n使用IDLE - 交互式环境(REPL) / 编写多行代码 / 运行程序 / 

In [195]:
df[0][1]

"['jackfrued / Python-100-Days']"

In [196]:
df.head()

,0,1
0,['microsoft / Terminal'],Welcome! This repository contains the source c...
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...


In [246]:
df.columns = ['title', 'original']
df.head()

,title,original
0,['microsoft / Terminal'],Welcome! This repository contains the source c...
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...


## Let's work on the columns for 'word counts'.  This is a loose term as we can visually see that some items that were originally hyphenated, and originally considered a single word, come back as multiple words.  Because of this, some readme files actually return more 'words' that the original text contained.

In [277]:
def original_word_count():
    df[['original_cnt']] = df[['original']]
    art_nums = list(range(df.original.count()))
    for art_num in art_nums:
        df.original_cnt[art_num] = len(df['original'][art_num].split())
    return df

In [278]:
original_word_count()
df.head()

,title,original,original_cnt
0,['microsoft / Terminal'],Welcome! This repository contains the source c...,1250
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...,741
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...,367
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...,46


In [280]:
df.original[4]

'\n\nBeautifully simple click-to-copy CSS effects\n\n\nhttps://cssfx.dev\n\nTo run locally, npm install then npm run dev.\nContributing\nCheck out the contributing guide to learn how you can help out!\nAlways be sure to follow the Code of Conduct.\nLicense\nThe CSSFX site and effects are MIT licensed.\n'

In [274]:
len(df['original'][0].split())


1250

In [281]:
def basic_clean():
    
    df[['article']] = df[['original']]
    art_nums = list(range(df.original.count()))
    for art_num in art_nums:
        original = df.article[art_num]
        article = re.sub(r'\s', ' ', original).lower()
        normalized = unicodedata.normalize('NFKD', article)
        just_ascii = normalized.encode('ascii', 'ignore')
        article = just_ascii.decode('utf-8')
        without_special_chars = re.sub(r'[^\w\s]', ' ', article)
        word_list = without_special_chars.split()
        article_final = ' '.join(word_list)
        df.article[art_num] = article_final
    return df

In [282]:
basic_clean().head()

,title,original,original_cnt,article
0,['microsoft / Terminal'],Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...


In [283]:
def article_word_count():
    df[['article_cnt']] = df[['article']]
    art_nums = list(range(df.original.count()))
    for art_num in art_nums:
        df.article_cnt[art_num] = len(df['article'][art_num].split())
    return df

In [284]:
article_word_count().head()

,title,original,original_cnt,article,article_cnt
0,['microsoft / Terminal'],Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50


In [286]:
df.shape

(23, 5)

In [287]:
df.dtypes

title           object
original        object
original_cnt    object
article         object
article_cnt     object
dtype: object

In [290]:
df['original_cnt']=df['original_cnt'].astype(int)
df['article_cnt']=df['article_cnt'].astype(int)
df.dtypes

title           object
original        object
original_cnt     int64
article         object
article_cnt      int64
dtype: object

In [293]:
df.isnull().sum()

title           0
original        0
original_cnt    0
article         0
article_cnt     0
dtype: int64

In [294]:
df[['article_per_change']] = df[['article_cnt']]
df[['article_per_change']] = df.article_cnt / df.original_cnt
df.head()

,title,original,original_cnt,article,article_cnt,article_per_change
0,['microsoft / Terminal'],Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303,1.042400
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380,1.035422
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50,1.086957


## Let's take a look at files that returned less than 70% of their original counts.  They do appear to have a lot of non-English characters as suspected.

In [298]:
df[df.article_per_change < .7]

,title,original,original_cnt,article,article_cnt,article_per_change
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577
8,['CyC2018 / CS-Notes'],\n离线阅读版本下载\n\n\n\n\n\n 算法 \n操作系统\n 网络 \n面向对象\n...,177,java offer leetcode linux http socket sql leet...,66,0.372881
15,['qianguyihao / Web'],项目介绍\n\n项目地址：https://github.com/qianguyihao/We...,18,https github com qianguyihao web web androidwe...,11,0.611111
19,['Snailclimb / JavaGuide'],点击订阅Java面试进阶指南(专为Java面试方向准备)。为什么要弄这个专栏?\n\n\n\...,290,java java special sponsors programmer advancem...,179,0.617241


In [302]:
def lemmatize():
    
    df[['lemmatized']] = df[['article']]
    art_nums = list(range(df.article.count()))
    wnl = nltk.stem.WordNetLemmatizer()
    for art_num in art_nums:
        lemm_article = df.lemmatized[art_num]
        lemmas = [wnl.lemmatize(word) for word in lemm_article.split()]
        article_lemmatized = ' '.join(lemmas)
        df.lemmatized[art_num] = article_lemmatized
    return df

In [303]:
lemmatize().head()

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,title,original,original_cnt,article,article_cnt,article_per_change,lemmatized
0,['microsoft / Terminal'],Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303,1.042400,welcome this repository contains the source co...
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577,python 100 python python c c python python jav...
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680,cs only chat a truly monstrous async web chat ...
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380,1.035422,overview powertoys is a set of utility for pow...
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50,1.086957,beautifully simple click to copy cs effect htt...


In [308]:
def lemm_word_count():
    df[['lemm_cnt']] = df[['article']]
    art_nums = list(range(df.original.count()))
    for art_num in art_nums:
        df.lemm_cnt[art_num] = len(df['lemmatized'][art_num].split())
        
    df.lemm_cnt = df.lemm_cnt.astype(int)
    return df

In [309]:
lemm_word_count().head()

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,title,original,original_cnt,article,article_cnt,article_per_change,lemmatized,lemm_cnt
0,['microsoft / Terminal'],Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303,1.042400,welcome this repository contains the source co...,1303
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577,python 100 python python c c python python jav...,519
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680,cs only chat a truly monstrous async web chat ...,783
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380,1.035422,overview powertoys is a set of utility for pow...,380
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50,1.086957,beautifully simple click to copy cs effect htt...,50


## The number of words stays the same for each file after lemmatization.  Some of the words have changed, CSS is now just CS.  

In [330]:
df[df.article_cnt != df.lemm_cnt]

,title,original,original_cnt,article,article_cnt,article_per_change,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_removed


## Let's remove stopwords and create two more columns, one for the count of words after stopwords removed and another than shows the percentage of words still left.

In [338]:
def remove_stopwards(extra_words=None, exclude_words=None):
    df = lemmatize()
    stopwords = nltk.corpus.stopwords.words('english')

    
    if extra_words != None:
        stopwords = stopwords + extra_words
    if exclude_words != None:
        stopwords = [word for word in stopwords if word not in exclude_words]
        
    
    df[['clean']] = df[['lemmatized']]
    df[['word_cnt_after_stop']] = df[['title']]

    art_nums = list(range(df.clean.count()))
    for art_num in art_nums:
        words = df.clean[art_num].split()
        without_stopwords = [word for word in words if word not in stopwords]
        df.word_cnt_after_stop[art_num] = len(without_stopwords)
        without_stopwords_join = ' '.join(without_stopwords)
        df.clean[art_num] = without_stopwords_join
    df[['per_stopwords_removed']] = df[['title']]
    df[['per_stopwords_removed']] = df.word_cnt_after_stop / df.lemm_cnt
    df[['per_stopwords_removed']] = df[['per_stopwords_removed']].astype(float)
    df[['word_cnt_after_stop']] = df[['word_cnt_after_stop']].astype(int)
    return df

In [339]:
remove_stopwards().head()

,title,original,original_cnt,article,article_cnt,article_per_change,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_removed
0,['microsoft / Terminal'],Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303,1.042400,welcome this repository contains the source co...,1303,welcome repository contains source code window...,803,0.616270
1,['jackfrued / Python-100-Days'],Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577,python 100 python python c c python python jav...,519,python 100 python python c c python python jav...,487,0.938343
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680,cs only chat a truly monstrous async web chat ...,783,cs chat truly monstrous async web chat using j...,408,0.521073
3,['microsoft / PowerToys'],Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380,1.035422,overview powertoys is a set of utility for pow...,380,overview powertoys set utility power user tune...,233,0.613158
4,['jolaleye / cssfx'],\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50,1.086957,beautifully simple click to copy cs effect htt...,50,beautifully simple click copy cs effect http c...,33,0.660000


In [340]:
df.describe()

,original_cnt,article_cnt,article_per_change,lemm_cnt,word_cnt_after_stop,per_stopwords_removed
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,716.739130,689.086957,0.965863,689.086957,467.478261,0.738458
std,702.887565,697.427671,0.264959,697.427671,436.985112,0.150487
min,18.000000,11.000000,0.372577,11.000000,11.000000,0.521073
25%,317.500000,363.000000,0.998607,363.000000,230.000000,0.627946
50%,373.000000,414.000000,1.026810,414.000000,369.000000,0.673184
75%,872.500000,768.000000,1.047406,768.000000,506.000000,0.856310
max,2978.000000,3057.000000,1.669355,3057.000000,1851.000000,1.000000


In [341]:
df[df.per_stopwords_removed < .6]

,title,original,original_cnt,article,article_cnt,article_per_change,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_removed
2,['kkuchta / css-only-chat'],\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680,cs only chat a truly monstrous async web chat ...,783,cs chat truly monstrous async web chat using j...,408,0.521073
21,['markphelps / flipt'],\n\n\nA feature flag solution that runs in you...,618,a feature flag solution that runs in your exis...,632,1.022654,a feature flag solution that run in your exist...,632,feature flag solution run existing infrastruct...,369,0.583861


In [342]:
labels = [['microsoft / Terminal', 'c++'],
 ['jackfrued / Python-100-Days', 'html'],
 ['kkuchta / css-only-chat', 'ruby'],
 ['microsoft / PowerToys', 'na'],
 ['jolaleye / cssfx', 'vue'],
 ['MisterBooo / LeetCodeAnimation', 'java'],
 ['flutter / flutter_web', 'dart'],
 ['TheAlgorithms / Python', 'python'],
 ['hiroppy / fusuma', 'css'],
 ['CyC2018 / CS-Notes', 'java'],
 ['jaywcjlove / linux-command', 'html'],
 ['flutter / flutter', 'dart'],
 ['996icu / 996.ICU', 'rust'],
 ['STVIR / pysot', 'na'],
 ['minamarkham / formation', 'shell'],
 ['azl397985856 / leetcode', 'javascript'],
 ['qianguyihao / Web', 'javascript'],
 ['microsoft / react-native-windows', 'c#'],
 ['sql-machine-learning / sqlflow', 'go'],
 ['sabakkps / backslide', 'na'],
 ['dgryski / go-perfbook', 'na'],
 ['Snailclimb / JavaGuide', 'java'],
 ['microsoft / vscode', 'typescript'],
 ['markphelps / flipt', 'go'],
 ['teoga / awesome-product-design', 'na']]

In [352]:
labels_df = pd.DataFrame(list(labels))
labels_df.head()

,0,1
0,microsoft / Terminal,c++
1,jackfrued / Python-100-Days,html
2,kkuchta / css-only-chat,ruby
3,microsoft / PowerToys,na
4,jolaleye / cssfx,vue


In [354]:
labels_df.columns = ['title', 'language']
labels_df.head()

,title,language
0,microsoft / Terminal,c++
1,jackfrued / Python-100-Days,html
2,kkuchta / css-only-chat,ruby
3,microsoft / PowerToys,na
4,jolaleye / cssfx,vue


In [360]:
df['title']= df.title.str.replace('[', '')
df['title'] = df.title.str.replace(']', '')
df.head()

,title,original,original_cnt,article,article_cnt,article_per_change,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_removed
0,'microsoft / Terminal',Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303,1.042400,welcome this repository contains the source co...,1303,welcome repository contains source code window...,803,0.616270
1,'jackfrued / Python-100-Days',Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577,python 100 python python c c python python jav...,519,python 100 python python c c python python jav...,487,0.938343
2,'kkuchta / css-only-chat',\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680,cs only chat a truly monstrous async web chat ...,783,cs chat truly monstrous async web chat using j...,408,0.521073
3,'microsoft / PowerToys',Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380,1.035422,overview powertoys is a set of utility for pow...,380,overview powertoys set utility power user tune...,233,0.613158
4,'jolaleye / cssfx',\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50,1.086957,beautifully simple click to copy cs effect htt...,50,beautifully simple click copy cs effect http c...,33,0.660000


In [361]:
df['title'] = df.title.str.replace('\'', '')
df.head()

,title,original,original_cnt,article,article_cnt,article_per_change,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_removed
0,microsoft / Terminal,Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303,1.042400,welcome this repository contains the source co...,1303,welcome repository contains source code window...,803,0.616270
1,jackfrued / Python-100-Days,Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577,python 100 python python c c python python jav...,519,python 100 python python c c python python jav...,487,0.938343
2,kkuchta / css-only-chat,\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680,cs only chat a truly monstrous async web chat ...,783,cs chat truly monstrous async web chat using j...,408,0.521073
3,microsoft / PowerToys,Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380,1.035422,overview powertoys is a set of utility for pow...,380,overview powertoys set utility power user tune...,233,0.613158
4,jolaleye / cssfx,\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50,1.086957,beautifully simple click to copy cs effect htt...,50,beautifully simple click copy cs effect http c...,33,0.660000


In [362]:
label_test = pd.merge(df, labels_df, on='title')
label_test

,title,original,original_cnt,article,article_cnt,article_per_change,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_removed,language
0,microsoft / Terminal,Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303,1.042400,welcome this repository contains the source co...,1303,welcome repository contains source code window...,803,0.616270,c++
1,jackfrued / Python-100-Days,Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577,python 100 python python c c python python jav...,519,python 100 python python c c python python jav...,487,0.938343,html
2,kkuchta / css-only-chat,\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680,cs only chat a truly monstrous async web chat ...,783,cs chat truly monstrous async web chat using j...,408,0.521073,ruby
3,microsoft / PowerToys,Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380,1.035422,overview powertoys is a set of utility for pow...,380,overview powertoys set utility power user tune...,233,0.613158,na
4,jolaleye / cssfx,\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50,1.086957,beautifully simple click to copy cs effect htt...,50,beautifully simple click copy cs effect http c...,33,0.660000,vue
5,flutter / flutter_web,\nWelcome to the code repository for Flutter f...,1424,welcome to the code repository for flutter for...,1492,1.047753,welcome to the code repository for flutter for...,1492,welcome code repository flutter web repository...,958,0.642091,dart
6,TheAlgorithms / Python,The Algorithms - Python \n\nAll algorithms imp...,2978,the algorithms python all algorithms implement...,3057,1.026528,the algorithm python all algorithm implemented...,3057,algorithm python algorithm implemented python ...,1851,0.605496,python
7,hiroppy / fusuma,\n\n\n\n📝 Make slides with Markdown easily.\n\...,568,make slides with markdown easily just write ma...,560,0.985915,make slide with markdown easily just write mar...,560,make slide markdown easily write markdown crea...,441,0.787500,css
8,CyC2018 / CS-Notes,\n离线阅读版本下载\n\n\n\n\n\n 算法 \n操作系统\n 网络 \n面向对象\n...,177,java offer leetcode linux http socket sql leet...,66,0.372881,java offer leetcode linux http socket sql leet...,66,java offer leetcode linux http socket sql leet...,63,0.954545,java
9,jaywcjlove / linux-command,\n\n\n\n\nLinux Command\n \n550 多个 Linux 命令，内容...,345,linux command 550 linux linux linux linuxde ne...,369,1.069565,linux command 550 linux linux linux linuxde ne...,369,linux command 550 linux linux linux linuxde ne...,366,0.991870,html


In [363]:
list(label_test)

['title',
 'original',
 'original_cnt',
 'article',
 'article_cnt',
 'article_per_change',
 'lemmatized',
 'lemm_cnt',
 'clean',
 'word_cnt_after_stop',
 'per_stopwords_removed',
 'language']

In [365]:
df = label_test[['title',
 'language',
 'original',
 'original_cnt',
 'article',
 'article_cnt',
 'article_per_change',
 'lemmatized',
 'lemm_cnt',
 'clean',
 'word_cnt_after_stop',
 'per_stopwords_removed']]

df.head()

,title,language,original,original_cnt,article,article_cnt,article_per_change,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_removed
0,microsoft / Terminal,c++,Welcome! This repository contains the source c...,1250,welcome this repository contains the source co...,1303,1.042400,welcome this repository contains the source co...,1303,welcome repository contains source code window...,803,0.616270
1,jackfrued / Python-100-Days,html,Python - 100天从新手到大师\n\n作者：骆昊\n\nPython应用领域和就业形...,1393,python 100 python python c c python python jav...,519,0.372577,python 100 python python c c python python jav...,519,python 100 python python c c python python jav...,487,0.938343
2,kkuchta / css-only-chat,ruby,\nCSS-Only Chat\nA truly monstrous async web c...,741,css only chat a truly monstrous async web chat...,783,1.056680,cs only chat a truly monstrous async web chat ...,783,cs chat truly monstrous async web chat using j...,408,0.521073
3,microsoft / PowerToys,na,Overview\nPowerToys is a set of utilities for ...,367,overview powertoys is a set of utilities for p...,380,1.035422,overview powertoys is a set of utility for pow...,380,overview powertoys set utility power user tune...,233,0.613158
4,jolaleye / cssfx,vue,\n\nBeautifully simple click-to-copy CSS effec...,46,beautifully simple click to copy css effects h...,50,1.086957,beautifully simple click to copy cs effect htt...,50,beautifully simple click copy cs effect http c...,33,0.660000


In [438]:
def get_article_text_with_labels():

    blobs = blob_list
    good_blobs = []
    read_me_list = []
    language_top = []
    
    for blob in blobs:
        blob_string=blob[0]
        blob_string=blob_string.replace(' ', '')
        
#         url_1 = 'https://github.com/' + str(blob_string) + '/blob/master/README.md'
        url_1 = 'https://github.com/' + str(blob_string)
        url = url_1
        language_url = 'https://github.com/' + str(blob)
        language_url = language_url.replace(' ', '')
        language_url = language_url.replace(']', '')
        language_url = language_url.replace('[', '')
        language_url = language_url.replace('\'', '')
        
        print(url)

        response = requests.get(url)
        language_response = requests.get(language_url)
        
        soup = BeautifulSoup(response.text)
#         language_soup = BeautifulSoup(language_response.text)
#         print(language_soup)
        
        
        read_me = soup.find(class_='markdown-body entry-content p-3 p-md-6')
        language_rm = soup.find('span', class_='lang')
#         read_me_list.append(str(blob) + ':')
        print(language_rm)
        
        if read_me != None:
        
            read_me_list.append(read_me.text)
            good_blobs.append(str(blob))
            language_top.append(str(language_rm))
        
#     return pd.DataFrame(list(zip(good_blobs, read_me_list)))

In [439]:
get_article_text_with_labels()

https://github.com/microsoft/Terminal
<span class="lang">C++</span>
https://github.com/jackfrued/Python-100-Days
<span class="lang">HTML</span>
https://github.com/kkuchta/css-only-chat
<span class="lang">Ruby</span>
https://github.com/microsoft/PowerToys
None
https://github.com/jolaleye/cssfx
<span class="lang">Vue</span>
https://github.com/MisterBooo/LeetCodeAnimation
<span class="lang">Java</span>
